In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *
from setting import *

In [ ]:
w = pd.read_table(PATH_DICT["w_file_path"], index_col=0)

h = pd.read_table(PATH_DICT["h_file_path"], index_col=0)

In [ ]:
if ELEMENT_ENTROPY_QUANTILE < 1:

    feature_entropy = w.apply(ccal.compute_entropy, axis=1)

    w = w.loc[
        feature_entropy.index[
            feature_entropy < feature_entropy.quantile(q=ELEMENTS_ENTROPY_QUANTILE)
        ]
    ]

    print(w.shape)

    sample_entropy = h.apply(ccal.compute_entropy)

    h = h[
        sample_entropy.index[
            sample_entropy < sample_entropy.quantile(q=ELEMENTS_ENTROPY_QUANTILE)
        ]
    ]

    print(h.shape)

In [ ]:
gps_map = ccal.GPSMap(
    w=w.T,
    h=h,
    w_pull_power=GPS_MAP_W_PULL_POWER,
    h_pull_power=GPS_MAP_H_PULL_POWER,
    plot=False,
)

In [ ]:
for w_or_h, hcc_k, bandwidth_factor, element_marker_size in (
    ("w", W_HCC_K, GPS_MAP_W_BANDWIDTH_FACTOR, GPS_MAP_W_ELEMENT_MARKER_SIZE),
    ("h", H_HCC_K, GPS_MAP_H_BANDWIDTH_FACTOR, GPS_MAP_H_ELEMENT_MARKER_SIZE),
):

    gps_map.plot_gps_map(
        w_or_h,
        title=TITLE,
        element_marker_size=element_marker_size,
        html_file_path="{}/map.html".format(
            PATH_DICT["{}_map_directory_path".format(w_or_h)]
        ),
    )

    hcc__k_x_column_file_path = PATH_DICT["{}_hcc__k_x_column_file_path".format(w_or_h)]

    if os.path.isfile(hcc__k_x_column_file_path):

        hcc__k_x_column = pd.read_table(hcc__k_x_column_file_path, index_col=0)

        gps_map.set_element_states(
            w_or_h,
            hcc__k_x_column.loc["K{}".format(hcc_k)],
            bandwidth_factor=bandwidth_factor,
        )

        gps_map.plot_gps_map(
            w_or_h,
            title=TITLE,
            element_marker_size=element_marker_size,
            html_file_path="{}/map_with_color.html".format(
                PATH_DICT["{}_map_directory_path".format(w_or_h)]
            ),
            plotly_file_path=PATH_DICT["{}_map_plotly_file_path".format(w_or_h)],
        )

In [ ]:
ccal.dump_gps_map(gps_map, PATH_DICT["gps_map_file_path"])